In [ ]:
from agent import agent
from board import board
from model import Net
# import torch.nn.functional as F
# import torch.optim as optim

In [ ]:
from agent import agent
from board import board
from moves import *
from model import Net
import torch
import copy
import numpy as np

# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def generate_trajectory(max_t=100):
        Agent=agent(copy.deepcopy(board))


        states=[]
        greedy_moves = []
        # predicted_moves=[]
        
        

        for t in range(max_t):
            
            state=Agent.board_to_array()
            states.append(state)
            # tensor = torch.from_numpy( state ).float().unsqueeze(0)
            # action=actor(tensor).detach().squeeze()
            # predicted_moves.append(action)
            tag,to,castles = Agent.sample("W")
            greedy_move = np.zeros(675)
            greedy_move[Agent.abs_to_rel_moves(tag,to,castles)]=1
            greedy_moves.append ( greedy_move )
            white_reward=Agent.make_move(tag,to,castles)


                
            if Agent.checkmate("B"):
                print("white won")
                break
            if Agent.stalemate("B"):
                

                break

            # add te obtained results to their relative lists ==> saved_log_probs, rewards, state_values
            tag,to,castles=Agent.sample("B")
            
            black_reward=Agent.make_move(tag,to,castles)
            
            if Agent.checkmate("W"):
                print("white won")
                break
            if Agent.stalemate("B"):
                print("black won")
                
                break

        return   states , greedy_moves 



In [ ]:
import pickle
with open('states.pkl', 'rb') as file:
    states = pickle.load(file)
with open('taraget.pkl', 'rb') as file:
    target = pickle.load(file)

In [ ]:
len(states)

In [ ]:
# states=[]
# target=[]
for i in range (200):
    print(i)
    a,b=generate_trajectory(100)
    states+=a
    target+=b

In [ ]:
import numpy as np
x=np.stack(states)
y=np.stack(target)
y.shape

In [ ]:


# # Save variable to a file
# with open('states.pkl', 'wb') as file:
#     pickle.dump(states, file)
# with open('taraget.pkl', 'wb') as file:
#     pickle.dump(target, file)

In [ ]:
import tensorflow as tf
import numpy as np

input1 = tf.keras.layers.Input(shape=(6,8,8))
# shape1 = tf.keras.layers.Reshape(target_shape=(8, 8, 6))(input1)
conv1 = tf.keras.layers.Conv2D(kernel_size=(8,8), padding="same", activation="relu", filters=64, input_shape=(8,8,1))(input1)
bn1 = tf.keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=1e-05)(conv1)
conv2 = tf.keras.layers.Conv2D(kernel_size=(8,8), padding="same", activation="relu", filters=64, input_shape=(8,8,1))(bn1)
bn2 = tf.keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=1e-05)(conv2)
flatten1 = tf.keras.layers.Flatten()(bn2)
# input2 = tf.keras.layers.Input(shape=(5,))

# conc = tf.keras.layers.concatenate([flatten1,input2])

Denselayer1 = tf.keras.layers.Dense(512, activation='relu')(flatten1)
# Denselayer2 = tf.keras.layers.Dense(512, activation='relu')(Denselayer1)
# Denselayer3 = tf.keras.layers.Dense(256, activation='relu')(Denselayer2)
# Denselayer4 = tf.keras.layers.Dense(256, activation='relu')(Denselayer3)
Output = tf.keras.layers.Dense(675, activation='softmax')(Denselayer1)



data_model = tf.keras.models.Model(inputs=input1, outputs=Output)

# predictions = data_model([(inputboard[:1]), (inputmeta[:1])]).numpy

metric =[tf.keras.metrics.CategoricalCrossentropy()]





#Removing the Clipnorm of clipnorm=1 may make training faster
#opt = tf.keras.optimizers.Adam(clipnorm=1)

opt = tf.keras.optimizers.Adam()




los = tf.keras.losses.CategoricalCrossentropy()

data_model.compile(optimizer=opt,
                   loss=los,
                   metrics=metric)
data_model.summary()


In [ ]:
los = tf.keras.losses.CategoricalCrossentropy()

data_model.compile(optimizer=opt,
                   loss=los,
                   metrics=metric)
data_model.summary()

In [ ]:
data_model.fit(x, y, epochs=100, batch_size=256, shuffle=True)

In [ ]:
# import random
# loss_hist=[]
# n=len(states)
# for i in range(10000):
# # Sample corresponding values from both lists using zip
#     sampled_indices = np.random.choice(n, 1, replace=False)
#     # print(sampled_indices)
# # Unzip the sampled pairs to get separate lists
#     sampled_state =np.stack(states)[sampled_indices]
#     sampled_target = np.stack(target)[sampled_indices]
#     sampled_state=torch.Tensor(sampled_state)
#     sampled_target=torch.Tensor(sampled_target)
#     pred=model(sampled_state.float())

#     loss=train(optimizer , pred , sampled_target)
#     if (i % 100)==0:print(f"loss in {i}th iteration is {loss}")
#     loss_hist.append(loss)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
loss_hist=[i.item() for i in loss_hist]
scores= pd.Series(loss_hist, name="scores_Actor")


In [ ]:
fig, ax = plt.subplots(1, 1)
_ = scores.plot(ax=ax, label="scores_Actor")
_ = (scores.rolling(window=100)
           .mean()
           .rename("Rolling Average")
           .plot(ax=ax))
ax.legend()
_ = ax.set_xlabel("Episode Number")
_ = ax.set_ylabel("scores_Actor")

In [ ]:
from agent import agent
from board import board
from moves import *
from model import Net
import torch
import copy
import torch.nn.functional as F
import torch.optim as optim

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def generate_trajectory(actor, max_t=100):
        Agent=agent(copy.deepcopy(board))

        # critic=Critic()
        saved_log_probs = []
        rewards = []
        state_values=[]
        # print("generate trajectory")
        
        

        for t in range(max_t):
            Agent.visualize()
            
            state=torch.tensor(Agent.board_to_array()).float().unsqueeze(0)
            mask,mp=Agent.rel_to_abs_moves()
            #state_val= critic(state)
            # try:
            action_id =  actor.forward(state)
            print(action_id)

            #      print('no')
            #      break
                
            # action=mp[action_id]





            # white_reward= Agent.make_move(action['tag'],action['to'],action['castles'])

            # saved_log_probs.append(log_prob)
            if Agent.checkmate("B"):
                rewards.append(30)
                print("white won")
                break
            if Agent.stalemate("B"):
                
                rewards.append(-2)
                break

            # add te obtained results to their relative lists ==> saved_log_probs, rewards, state_values
            tag,to,castles=Agent.sample("B")
            
            black_reward=Agent.make_move(tag,to,castles)
            
            if Agent.checkmate("W"):
                rewards.append(-30)
                print("black won")
                break
            if Agent.stalemate("W"):
                
                rewards.append(2)
                break

            # add code here
            
            # # add code here
            # rewards.append(white_reward - black_reward)
           
            # add code here

        # Agent.visualize()
        return  saved_log_probs , rewards , state_values



In [ ]:
Agent=agent(board)
Agent.visualize()
pred=model(torch.tensor(Agent.board_to_array()).float().unsqueeze(0)).squeeze().detach().numpy()
print(pred)
id=np.argmax(pred)
_,a=Agent.rel_to_abs_moves(color="W")
a[id]

In [ ]:
sampled_indices = np.random.choice(5, size=2, replace=False)
np.stack([5,2,1,2,3,5,4,5])[sampled_indices ]

In [ ]:
ord('9')